<a href="https://colab.research.google.com/github/samas-it-services/open-course-delta-lake/blob/master/Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Delta Lake Quick Start
This notebook provides a quick start guide to using Delta Lake, as outlined in the [Delta Lake Documentation](https://docs.delta.io/latest/quick-start.html).


In [2]:
# Install necessary dependencies
!pip install pyspark==3.3.1
!pip install delta-spark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 23.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845493 sha256=cc533cfab430c4a9d63340b0bb9e79fe65d83fda18fc58a03d1995bba474d6c4
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 20.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e79929f8901ac1b47b1d24af2b18ed3b1d222db8304b3a

In [3]:
# Import Libraries
from pyspark.sql import SparkSession
from delta import *


In [4]:
# Create a Spark Session with Delta Lake
builder = SparkSession.builder.appName("DeltaLakeQuickStart") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [5]:
# Create a Delta Table
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")


In [7]:
# Read from the Delta Table
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()


+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



In [8]:
# Update the Delta Table
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")


In [9]:
# Read the Updated Delta Table
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()


+---+
| id|
+---+
|  7|
|  8|
|  9|
|  5|
|  6|
+---+



Conditional update without overwrite
Delta Lake provides programmatic APIs to conditional update, delete, and merge (upsert) data into tables. Here are a few examples.

In [10]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



You should see that some of the existing rows have been updated and new rows have been inserted.

For more information on these operations, see Table deletes, updates, and merges.



# Read older versions of data using time travel
You can query previous snapshots of your Delta table by using time travel. If you want to access the data that you overwrote, you can query a snapshot of the table before you overwrote the first set of data using the versionAsOf option.





In [11]:
df = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



You should see the first set of data, from before you overwrote it. Time travel takes advantage of the power of the Delta Lake transaction log to access data that is no longer in the table. Removing the version 0 option (or specifying version 1) would let you see the newer data again. For more information, see Query an older snapshot of a table (time travel).


# Write a stream of data to a table
You can also write to a Delta table using Structured Streaming. The Delta Lake transaction log guarantees exactly-once processing, even when there are other streams or batch queries running concurrently against the table. By default, streams run in append mode, which adds new records to the table:

In [12]:
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "/tmp/checkpoint").start("/tmp/delta-table")

While the stream is running, you can read the table using the earlier commands.



# stop the stream
You can stop the stream by running stream.stop() in the same terminal that started the stream.

For more information about Delta Lake integration with Structured Streaming, see Table streaming reads and writes. See also the Structured Streaming Programming Guide on the Apache Spark website.



In [ ]:
stream.stop()

# Read a stream of changes from a table
While the stream is writing to the Delta table, you can also read from that table as streaming source. For example, you can start another streaming query that prints all the changes made to the Delta table. You can specify which version Structured Streaming should start from by providing the startingVersion or startingTimestamp option to get changes from that point onwards. See Structured Streaming for details.


In [13]:
stream2 = spark.readStream.format("delta").load("/tmp/delta-table").writeStream.format("console").start()
